In [11]:
import numpy as np
import cv2

In [12]:
def findCircles(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Blur the image to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply the Hough Transform to find circles
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=70, minRadius=5, maxRadius=200)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the outer circle
            cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
            # Draw the center of the circle
            cv2.circle(frame, (i[0], i[1]), 2, (255, 0, 0), 3)

    cv2.imshow("Webcam", frame) # This will open an independent window
    return circles

In [13]:
def recognizeRed(frame):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Define range of red color in HSV
    lower_red = np.array([-10, 50, 50])
    upper_red = np.array([10, 255, 255])
    # Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("Filtered", res) # This will open an independent window
    return res

In [14]:
def maskCircle(res, center, radius):
    # Create a black mask with the same shape as the image
    mask = np.zeros_like(res)

    # Create a white circle on the mask
    cv2.circle(mask, center, radius, (255, 255, 255), -1)

    # Apply the mask to the image
    masked_image = cv2.bitwise_and(res, mask)

    # cv2.imshow("Masked", masked_image) # This will open an independent window

    # cv2.waitKey(0)

    return masked_image

In [15]:
def countRedPixels(res, circles):
    counts=np.array([])

    # Created a mask for each circle
    for circle in circles[0, :]:
        center = (circle[0], circle[1])
        radius = circle[2]
        masked_image = maskCircle(res, center, radius)
        #cv2.imshow("Masked", masked_image) # This will open an independent window

        gray_masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
        
        # Count the number of non-black pixels
        count = cv2.countNonZero(gray_masked_image)
        counts = np.append(counts, count)

        #print(count)
        #cv2.waitKey(0)
    
    return counts

In [16]:
def findMostRedCircle(counts, circles):
    # Find the circle with the most red pixels per area
    radii = circles[0, :, 2]
    max_index = np.argmax(counts/radii**2)
    max_circle = circles[0, max_index]
    #print(max_circle)
    #cv2.waitKey(0)
    return max_circle

In [17]:
def recognizeOOI(cap):
    success, frame = cap.read()

    circles=findCircles(frame)

    if circles is not None:
        #print(circles)
        res = recognizeRed(frame)
        counts = countRedPixels(res, circles)
        max_circle = findMostRedCircle(counts, circles)
        #print(max_circle)
        #print('\n')
        return max_circle
    
    return None

In [18]:
def circleInFieldOfRegard(ref_circle, circles):
    # Get the dimensions of the field of regard
    maxdist=20

    # Get the coordinates of the ref_circle
    xref = ref_circle[0]
    yref = ref_circle[1]

    goodcircles = np.copy(circles)

    #print(circles)
    i=0
    delindex = []

    for circle in circles[0, :]:
        x = circle[0]
        y = circle[1]
        if (xref-x)**2+(yref-y)**2>maxdist**2:
            #print((xref-x)**2+(yref-y)**2)
            delindex.append(i)
        i+=1

    goodcircles = np.delete(goodcircles, delindex, axis=1)

    if(goodcircles.size==0):
        return None

    #print(goodcircles)
    #print(circles)
    #cv2.waitKey(0)
    return goodcircles

In [19]:
def trackOOI(ref_circle, cap):
    # # Create a VideoCapture object
    # cap = cv2.VideoCapture(0)

    lastcircle = ref_circle
    failcount = 0
    max_circle = None

    while True:
        # Capture frame-by-frame
        success, frame = cap.read()

        max_circle = None

        circles=findCircles(frame)

        if circles is not None:
            #print(circles)
            goodcircles = circleInFieldOfRegard(lastcircle, circles)
            if goodcircles is not None:
                #print("if")
                res = recognizeRed(frame)
                counts = countRedPixels(res, goodcircles)
                max_circle = findMostRedCircle(counts, goodcircles)           
                #print('\n')
                lastcircle= max_circle
                failcount = 0
            else:
                #print("else")
                failcount += 1
                print(failcount)
                if failcount>10:
                    break
        else:
            failcount += 1
            print(failcount)
            if failcount>10:
                break
        
        #print(failcount)
        print(max_circle)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


In [20]:
import cv2

cap = cv2.VideoCapture(0)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height

while True:
    #success, img = cap.read()

    # Do some processing on the image here (using opencv) - inspiration below:
    # https://www.geeksforgeeks.org/detect-an-object-with-opencv-python/
    # https://docs.opencv.org/4.x/da/d53/tutorial_py_houghcircles.html

    print("Recognizing OOI")

    max_circle=recognizeOOI(cap)

    print(max_circle)

    if max_circle is not None:
        print("Tracking OOI")
        # print(max_circle)
        # print('\n')
        # cv2.waitKey(0)
        trackOOI(max_circle, cap)
        
    #cv2.imshow("Webcam", img) # This will open an independent window
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
        
cv2.destroyAllWindows() 
cv2.waitKey(1) # normally unnecessary, but it fixes a bug on MacOS where the window doesn't close

Recognizing OOI
[220 220  29]
Tracking OOI
[218 226  27]
[300 228  29]
[302 220  30]
[218 224  25]


C:\Users\s194149\AppData\Local\Temp\ipykernel_7312\3156998007.py:18: RuntimeWarning: overflow encountered in scalar subtract
  if (xref-x)**2+(yref-y)**2>maxdist**2:


[218 226  27]
[218 224  25]
[216 226  26]
[218 224  25]
[216 226  26]
[218 224  26]
[218 226  27]
[220 220  32]
[222 222  27]
[224 220  30]
[222 224  25]
[226 220  33]
[226 224  26]
[228 224  25]
[232 224  29]
[234 222  28]
[234 224  29]
[234 224  28]
[234 226  27]
[236 222  27]
[236 222  27]
[236 222  27]
[236 222  27]
[236 222  27]
[238 222  28]
[238 218  33]
[240 220  31]
[234 226  27]
[234 226  26]
[238 222  28]
[234 228  29]
[240 220  31]
[236 222  30]
[238 222  28]
[238 224  28]
[322 232  27]
[320 228  27]
[314 230  32]
[314 230  32]
[320 232  27]
[318 228  27]
[318 228  27]
[318 228  28]
[318 230  27]
[318 228  28]
[316 228  29]
[320 228  29]
[318 230  27]
[318 228  28]
[320 234  34]
[318 228  30]
[316 228  28]
[316 228  28]
[318 232  29]
[318 228  28]
[320 232  27]
[312 232  32]
[318 232  29]
[318 228  28]
[316 228  28]
[318 232  29]
[318 230  27]
[320 228  30]
[318 230  27]
[322 236  33]
[318 228  29]
[318 228  28]
[318 228  28]
[320 234  27]
[314 226  36]
[318 228  28]
[318 2

-1